In [1]:
from IPython.core.display import display, HTML
from h5py import File
import pandas as pd
import numpy as np
from pathlib import Path
import os
root_path = Path(r"C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche")
print(root_path)
os.chdir(str(root_path))
from FIBI.data.analyse_cpp_results.tsp.extract_infos import parse_key, parse_metadata, DataCategory, corresp_impr_class
import json

C:\Users\robin\AppData\Local\Temp\ipykernel_27720\1494291354.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche


## Load data

In [2]:
data = []
with File("data/table1_exp_impr.hdf5", "r") as f:
    for k,v in f["metadata"].items():
        dico_key = parse_key(k)
        v = np.copy(v)
        dico_value = {"key_dataset":k,**parse_metadata(v)}
        data.append(dico_value)
df = pd.DataFrame(data)


In [3]:
display(df.head())

,key_dataset,SEED_POINTS,SEED_ASSIGN,NUM_CLUST,NUM_POINTS,IMPR_CLASS,IT_ORDER,init_cost,final_cost,num_iter,num_iter_glob,duration
0,"100,0,6225993,32000,1,32000,1,0",32000,32000,6225993,100,BICL,CURR,1020.768756,850.823880,26,128648,0.003968
1,"100,0,6225993,32001,1,32001,1,0",32001,32001,6225993,100,BICL,CURR,911.616558,760.150810,18,89064,0.002830
2,"100,0,6225993,32002,1,32002,1,0",32002,32002,6225993,100,BICL,CURR,957.736873,792.073222,17,84116,0.002618
3,"100,0,6225993,32003,1,32003,1,0",32003,32003,6225993,100,BICL,CURR,950.590694,838.215907,14,69272,0.002196
4,"100,0,6225993,32004,1,32004,1,0",32004,32004,6225993,100,BICL,CURR,918.930432,804.859964,19,94012,0.003604


## Verify that we have all of the required results

In [4]:
def check_print(cond:bool):
    if cond:
        print("✅")
    else:
        print("❌")

Number of points:

Expected: [20,30,40,50,60,70,80,90,100,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]

Found:

In [5]:
expected = [20,30,40,50,60,70,80,90,100,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]
num_points = list(df[str(DataCategory.NUM_POINTS)].unique())
num_points.sort()
print(num_points)
check_print(set(num_points) == set(expected))

[20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000]
✅


Number of different point and assign seed per number of point x algorithm

Expected:

- for n in [20, 30, ..., 150] : 1000
- for n in [200, 250, ..., 500] : 100
- for n in [500, 600, ..., 1000] : 100

Found:

In [6]:
groups_counts = df.groupby([str(DataCategory.NUM_POINTS),str(DataCategory.IMPR_CLASS)]).count().reset_index()
num_points = [20,30,40,50,60,70,80,90,100,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]
expected = {n: 1000 if n<=150 else 100 for n in num_points}
cond = True

for n in expected:
    for impr_class in corresp_impr_class.values():
        df_tmp = groups_counts.query(f"{str(DataCategory.NUM_POINTS)} == {n} and {str(DataCategory.IMPR_CLASS)} == '{impr_class}'")
        cond = cond and df_tmp.iloc[0][str(DataCategory.SEED_POINTS)] == expected[n]
        cond = cond and df_tmp.iloc[0][str(DataCategory.SEED_ASSIGN)] == expected[n]
check_print(cond)

✅


## Statistics

### Computed statistics

In [12]:
df_table1_src = df[[
    str(DataCategory.NUM_POINTS),
    str(DataCategory.IMPR_CLASS),
    str(DataCategory.final_cost),
    str(DataCategory.num_iter),
    str(DataCategory.num_iter_glob),
    str(DataCategory.duration),
    ]]
df_table1 = df_table1_src.copy().groupby([str(DataCategory.NUM_POINTS),str(DataCategory.IMPR_CLASS)]).mean().reset_index()
# df_table1 = df_table1.query(f"{str(DataCategory.IMPR_CLASS)} != 'BICL'")
df_table1.sort_values(by=[str(DataCategory.NUM_POINTS),str(DataCategory.IMPR_CLASS)][::-1], inplace=True)
df_table1.reset_index(drop=True, inplace=True)
df_table1_str = df_table1.to_html(formatters={
    str(DataCategory.NUM_POINTS): '{:d}'.format,
    str(DataCategory.IMPR_CLASS): '{:s}'.format,
    str(DataCategory.final_cost): '{:.2f}'.format,
    str(DataCategory.num_iter): '{:.0f}'.format,
    str(DataCategory.num_iter_glob): '{:.0f}'.format,
    str(DataCategory.duration): '{:.4e}'.format,
})
display(HTML(df_table1_str))

,NUM_POINTS,IMPR_CLASS,final_cost,num_iter,num_iter_glob,duration
0,20,BICL,388.26,5,984,2.6920e-05
1,30,BICL,468.46,7,3189,8.9210e-05
2,40,BICL,531.70,9,7153,2.0370e-04
3,50,BICL,588.97,11,13540,3.7478e-04
4,60,BICL,638.79,13,22881,7.0366e-04
5,70,BICL,685.20,15,35942,1.1434e-03
6,80,BICL,731.94,17,52120,1.5316e-03
7,90,BICL,773.81,19,74252,2.5888e-03
8,100,BICL,811.50,20,98450,3.5047e-03
9,110,BICL,847.78,22,129868,3.6954e-03


## Export results for visualization

In [8]:
computed = df_table1_src.copy()

computed = computed[["NUM_POINTS","IMPR_CLASS","final_cost","num_iter","duration"]]
computed["source"] = "computed"
new_df = computed.copy()
display(new_df)

,NUM_POINTS,IMPR_CLASS,final_cost,num_iter,duration,source
0,100,BICL,850.823880,26,0.003968,computed
1,100,BICL,760.150810,18,0.002830,computed
2,100,BICL,792.073222,17,0.002618,computed
3,100,BICL,838.215907,14,0.002196,computed
4,100,BICL,804.859964,19,0.003604,computed
...,...,...,...,...,...,...
60795,900,FINL,2347.192066,285,0.158043,computed
60796,900,FINL,2368.535694,225,0.150628,computed
60797,900,FINL,2307.361166,308,0.156223,computed
60798,900,FINL,2304.615974,256,0.176032,computed


In [9]:
def q1(x):
    return x.quantile(0.25)

def median(x):
    return x.quantile(0.50)
def q3(x):
    return x.quantile(0.75)
f = ["mean","std","min",q1,median,q3,"max"]
df_avg = new_df.groupby(["NUM_POINTS","IMPR_CLASS","source"]).agg(f)
df_avg.reset_index(inplace=True)
df_avg.fillna(-1,inplace=True)
Lout = []
for elem in df_avg.to_dict(orient="records"):
    dico = {}
    for k,v in elem.items():
        if k[1] == '':
            dico[k[0]] = v
        else:
            if k[0] not in dico:
                dico[k[0]] = {}
            dico[k[0]][k[1]] = v
    dico["INIT_CHOICE"] = "greedy"
    Lout.append(dico)
dico_filtered = {}
for n in new_df["NUM_POINTS"].unique():
    for impr_class in new_df["IMPR_CLASS"].unique():
        for source in new_df["source"].unique():
            tmp = new_df.query(f"NUM_POINTS == {n} and IMPR_CLASS == '{impr_class}' and source == '{source}'").copy()
            tmp_dico = tmp.to_dict(orient="records")
            for i in range(len(tmp_dico)):
                tmp_dico[i]["INIT_CHOICE"] = "greedy"
            if len(tmp_dico) == 0:
                continue
            dico_filtered[f"{n}_{impr_class}_{source}"] = tmp_dico
            
dico_out = {"grouped":Lout,"filtered":dico_filtered}

with open("data/table1_impr_visualize.json","w") as f:
    json.dump(dico_out,f,indent=2)

In [13]:
from itertools import product
data = {
    ('NUM_POINTS',''):[],
    ('Obj. function values','BICL'):[],
    ('Obj. function values','BINL'):[],
    ('Obj. function values','FICL'):[],
    ('Obj. function values','FINL'):[],
    ('CPU times','BICL'):[],
    ('CPU times','BINL'):[],
    ('CPU times','FICL'):[],
    ('CPU times','FINL'):[],
    ('# of iterations','BICL'):[],
    ('# of iterations','BINL'):[],
    ('# of iterations','FICL'):[],
    ('# of iterations','FINL'):[],
}
nPts = df_table1["NUM_POINTS"].unique()
imprClass = df_table1["IMPR_CLASS"].unique()
attribute = [["final_cost",'Obj. function values'],["duration",'CPU times'],["num_iter",'# of iterations']]
for npt,impr in product(nPts,imprClass):
    tmp_df = df_table1.query(f"NUM_POINTS == {npt} and IMPR_CLASS == '{impr}'")
    try:
        dico = tmp_df.to_dict(orient="records")[0]
    except Exception:
        continue
    if npt not in data[('NUM_POINTS','')]:
        data[('NUM_POINTS','')].append(npt)
    for name,label in attribute:
        data[(label,impr)].append(float(f'{dico[name]:.2f}'))
# print([len(data[k]) for k in data])
pd.set_option('display.max_rows', None)
df_as_paper = pd.DataFrame(data)
df_as_paper[('% impr.','FICL')] = ((df_as_paper[('Obj. function values','BINL')]-df_as_paper[('Obj. function values','FICL')])/df_as_paper[('Obj. function values','BINL')]*100).round(2)
df_as_paper[('% impr.','FINL')] = ((df_as_paper[('Obj. function values','BINL')]-df_as_paper[('Obj. function values','FINL')])/df_as_paper[('Obj. function values','BINL')]*100).round(2)
cols = df_as_paper.columns.tolist()
df_as_paper = df_as_paper[cols[:4]+cols[-2:]+cols[5:-2]]
display(df_as_paper)
pd.set_option('display.max_rows', 20)

NUM_POINTS Obj. function values                   % impr.       CPU times  \
                              BICL     BINL     FICL    FICL  FINL      BICL   
0          20               388.26   390.30   394.04   -0.96 -0.40      0.00   
1          30               468.46   466.96   474.05   -1.52 -1.20      0.00   
2          40               531.70   533.83   539.89   -1.14 -1.07      0.00   
3          50               588.97   590.45   596.35   -1.00 -0.66      0.00   
4          60               638.79   642.07   651.12   -1.41 -0.87      0.00   
5          70               685.20   688.27   699.28   -1.60 -0.98      0.00   
6          80               731.94   733.01   743.32   -1.41 -1.00      0.00   
7          90               773.81   772.04   782.47   -1.35 -1.05      0.00   
8         100               811.50   811.08   822.28   -1.38 -1.01      0.00   
9         110               847.78   849.60   860.03   -1.23 -0.84      0.00   
10        120               883.70   884.11   896.06   -1.35 -1.10      0.00   
11        130               915.22   916.81   929.06   -1.34 -1.35      0.01   
12        140               949.14   951.85   962.98   -1.17 -0.70      0.01   
13        150               980.90   981.72   995.08   -1.36 -0.91      0.01   
14        200              1122.53  1125.13  1138.64   -1.20 -0.59      0.02   
15        250              1245.08  1253.42  1266.63   -1.05 -0.80      0.04   
16        300              1363.49  1364.72  1378.30   -1.00 -0.81      0.06   
17        350              1466.71  1468.40  1493.50   -1.71 -0.94      0.10   
18        400              1564.95  1565.71  1583.29   -1.12 -0.99      0.16   
19        450              1653.84  1655.79  1679.19   -1.41 -0.65      0.22   
20        500              1748.30  1744.87  1763.38   -1.06 -0.63      0.34   
21        600              1899.93  1907.19  1927.81   -1.08 -0.79      0.57   
22        700              2052.00  2052.94  2077.68   -1.21 -0.58      0.93   
23        800              2184.67  2190.28  2219.17   -1.32 -0.65      1.49   
24        900              2316.50  2311.71  2346.45   -1.50 -0.99      2.10   
25       1000              2441.58  2442.64  2471.66   -1.19 -1.01      3.04   

                     # of iterations                          
    BINL  FICL  FINL            BICL    BINL    FICL    FINL  
0   0.00  0.00  0.00            5.23    5.11    7.54    6.52  
1   0.00  0.00  0.00            7.37    7.08   11.61    9.55  
2   0.00  0.00  0.00            9.19    8.97   14.99   12.06  
3   0.00  0.00  0.00           11.07   10.72   18.78   15.56  
4   0.00  0.00  0.00           12.94   12.19   22.43   17.69  
5   0.00  0.00  0.00           14.89   14.28   25.47   21.25  
6   0.00  0.00  0.00           16.50   16.04   28.67   24.24  
7   0.00  0.00  0.00           18.55   17.53   32.62   26.96  
8   0.00  0.00  0.00           19.90   19.21   35.90   29.93  
9   0.00  0.00  0.00           21.67   20.93   39.12   32.70  
10  0.00  0.00  0.00           23.45   22.61   42.97   35.02  
11  0.00  0.01  0.00           24.98   24.08   46.08   38.08  
12  0.00  0.01  0.00           26.87   25.64   49.34   40.62  
13  0.00  0.01  0.00           28.26   27.32   52.71   43.80  
14  0.01  0.02  0.01           36.02   34.85   67.33   56.12  
15  0.01  0.04  0.01           44.70   42.86   87.10   72.24  
16  0.02  0.07  0.01           52.29   50.54  100.80   82.99  
17  0.02  0.11  0.02           60.81   57.89  118.17   95.66  
18  0.03  0.16  0.03           69.27   65.51  136.14  108.08  
19  0.04  0.26  0.03           75.65   72.66  149.83  123.96  
20  0.05  0.36  0.04           84.38   79.56  166.29  137.97  
21  0.07  0.63  0.06           98.37   92.88  198.02  168.08  
22  0.10  1.05  0.09          115.91  107.32  231.79  185.85  
23  0.13  1.63  0.11          129.18  122.74  265.74  218.91  
24  0.18  2.34  0.16          146.37  136.58  291.13  246.24  
25  0.21  2.97  0.24          158.04  151.57  319.33  269.41